In [1]:
from glob import glob

In [2]:
imgs = glob("static/imgs/*.jpg")

In [27]:
import numpy as np

def get_files(imgs, n_targets=66, n_filler=22, n_vigilence=12):
    files = np.random.choice(imgs, n_targets + n_filler + n_vigilence, replace=False)
    return files[:n_targets], files[n_targets: n_targets + n_filler], files[n_targets + n_filler: n_targets + n_filler + n_vigilence]
    
def next_vig(available, i, max_vigilence):
    l = []
    for j in range(i+1, len(available)):
        if available[j] - available[i] > 3:
            if available[j] - available[i] <= max_vigilence:
                l.append(j)
            else:
                break
    if len(l) == 0:
        return None
    return np.random.choice(l, 1)[0]

def next_target(labels, target, target_gap):
    temp_gap = 0; this_gap = 0
    while True:
        if labels[target + target_gap + temp_gap] == 0:
            this_gap = target_gap + temp_gap
            break
        if labels[target + target_gap - temp_gap] == 0:
            this_gap = target_gap - temp_gap
            break
        temp_gap += 1
    return this_gap

def set_target_reps(labels, files,file_targets, targets, target_gap):
    for i, target in enumerate(targets):  
        this_gap = next_target(labels, target, target_gap)
        labels[target+this_gap] = 2
        files[target+this_gap] = file_targets[i]

def get_sequence(file_targets, file_filler, file_vigilence, target_gap=80, max_vigilence=10):
    label_list = ["filler", "target", "target_rep", "vigilence", "vigilence_rep"]
    n_targets, n_filler, n_vigilence = len(file_targets), len(file_filler), len(file_vigilence)
    # shuffle
    np.random.shuffle(file_targets)
    np.random.shuffle(file_filler)
    np.random.shuffle(file_vigilence)
    n = n_targets*2 + n_filler + n_vigilence*2
    labels = np.zeros(n)
    files = np.array([file_targets[0]]*n)
    targets = np.random.choice(n-target_gap, n_targets, replace=False)
    labels[targets] = 1
    files[targets] = file_targets
    set_target_reps(labels, files,file_targets, targets, target_gap)
    available = np.array(range(n))[labels == 0]
    current_vigilence = 0
    for i in range(n_vigilence*2):
        s = np.random.choice(len(available), 1)[0]
        s_next = next_vig(available, s, max_vigilence)
        if s_next is None:
            continue
        labels[available[s]] = 3  
        files[available[s]] = file_vigilence[current_vigilence]
        labels[available[s_next]] = 4
        files[available[s_next]] = file_vigilence[current_vigilence]
        available = np.delete(available, s_next)
        available = np.delete(available, s)
        current_vigilence += 1
        if current_vigilence == n_vigilence:
            break

    files[available] = file_filler
    return files, labels

def evaluation(labels, answers):
    labels = np.array(labels)
    answers = np.array(answers)
    results = {}
    results["correct_filler"] = sum(answers[labels==0]==0)
    results["correct_target"] = sum(answers[labels==1]==0)
    results["correct_target_rep"] = sum(answers[labels==2]==1)
    results["correct_vigilence"] = sum(answers[labels==3]==0)
    results["correct_vigilence_rep"] = sum(answers[labels==4]==1)
    return results

def print_result(n_targets, n_filler, n_vigilence, labels, answers):
    e = evaluation(labels, answers)
    n_targets, n_filler, n_vigilence = len(file_targets), len(file_filler), len(file_vigilence)
    print("filler accuracy: {:.1f}%, target accuracy: {:.1f}%, vigilence accuracy: {:.1f}%".format(e['correct_filler']/len(file_filler)*100, 
                                                                                               (e['correct_target']+e['correct_target_rep'])/(2*len(file_targets))*100, 
                                                                                               (e['correct_vigilence']+e['correct_vigilence_rep'])/(2*len(file_vigilence))*100))

In [32]:
n_targets, n_filler, n_vigilence = 66, 44, 12
file_targets, file_filler, file_vigilence = get_files(imgs, n_targets=n_targets, n_filler=n_filler, n_vigilence=n_vigilence)
files, labels = get_sequence(file_targets, file_filler, file_vigilence)

In [12]:
answers = np.random.rand(len(labels))
answers[answers>0.5] = 1
answers[answers<=0.5] = 0

In [15]:
e = evaluation(labels, answers)

In [25]:
print_result(n_targets, n_filler, n_vigilence, labels, answers)

filler accuracy: 72.7%, target accuracy: 51.5%, vigilence accuracy: 20.8%


In [31]:
len( [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

198

In [1]:
a = {1:2}

In [4]:
list(a.keys())[0]

1

In [6]:
for k, v in a.items():
    print("{} {}".format(k, v))

1 2


In [19]:
from flask import Flask
from flask_login_multi.login_manager import LoginManager   

app=Flask(__name__)

login_manager = LoginManager(app)   

login_manager.blueprint_login_views = {  
        'user':  "user.user_login",  
        'admin': "admin.admin_login",  
    }  

ModuleNotFoundError: No module named 'flask_login_multi'

In [3]:
import pickle, os
import numpy as np
[evaluation, scores, marks] = pickle.load(open(os.path.join("data/ji", "data.pkl"), 'rb'))

In [4]:
evaluation

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [5]:
evaluation.append([1,2])

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, [1, 2]]